In [1]:
from __future__ import print_function
import keras
from keras.layers import Dense, Conv2D,BatchNormalization, Activation
from keras.layers import AveragePooling2D, Input, Flatten
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras.callbacks import ReduceLROnPlateau
from keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from keras.regularizers import l2
from keras import backend as K
from keras.models import Model
from keras.datasets import cifar10

import numpy as np
import os

In [2]:
batch_size=32
epochs=20
data_augmentation=True
num_classes=10

subtract_pixel_mean=True

In [3]:
n=6
version=2

if version ==1:
    depth =n*6+2
elif version==2:
    depth=n*9+2

model_type=f"ResNet{depth}v{version}"


In [4]:
model_type

'ResNet56v2'

In [5]:
(x_train,y_train), (x_test,y_test)=cifar10.load_data()
input_shape=x_train.shape
print(input_shape)

170498071/170498071 ━━━━━━━━━━━━━━━━━━━━ 744s 4us/step
(50000, 32, 32, 3)


In [6]:
input_shape=x_train.shape[1:]
x_train=x_train.astype('float32')/255
x_test=x_test.astype('float32')/255

if subtract_pixel_mean:
    x_train_mean=np.mean(x_train, axis=0)
    x_train-=x_train_mean
    x_test-=x_train_mean

print("x_train shape:", x_train.shape)
print(x_train.shape[0],"train samples")
print(x_test.shape[0],"test samples")
print("y_train shape:",y_train.shape)

x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples
y_train shape: (50000, 1)


In [7]:
#One hot encoding
y_train=keras.utils.to_categorical(y_train,num_classes)
y_test=keras.utils.to_categorical(y_test,num_classes)

In [8]:
def lr_schedule(epoch):
    lr=1e-3
    if epoch >= 80:
        lr *= 0.5e-3
    elif epoch >= 60:
        lr *= 1e-3
    elif epoch >= 40:
        lr *= 1e-2
    elif epoch >= 20:
        lr *= 1e-1
    print('Learning rate: ', lr)
    return lr

In [9]:
def resnet_layer(
        inputs,
        num_filters=16,
        kernel_size=3,
        strides=1,
        activation='relu',
        batch_normalization=True,
        conv_first=True):
    conv=Conv2D(num_filters,kernel_size=kernel_size,strides=strides,padding="same",
                kernel_initializer='he_normal', kernel_regularizer=l2(1e-4))
    x=inputs
    if conv_first:
        x=conv(x)
        if batch_normalization:
            x=BatchNormalization()(x)
        if activation is not None:
            x=Activation(activation)(x)
    else:
        if batch_normalization:
            x=BatchNormalization()(x)
        if activation is not None:
            x=Activation(activation)(x)
        x=conv(x)
    return x


In [14]:
def resnet_v1(input_shape,depth,num_classes=10):
    if (depth - 2) % 6!= 0:
        raise ValueError("depth should be 6n+2 (where n is a non-negative integer).")
    num_filters = 16
    num_res_blocks = int((depth - 2) / 6)
    inputs=Input(shape=input_shape)
    x=resnet_layer(inputs=inputs)

    for stack in range(3):
        for res_block in range(num_res_blocks):
            strides=1
            if stack > 0 and res_block == 0:
                strides=2
            y=resnet_layer(inputs=x, num_filters=num_filters, strides=strides)
            y=resnet_layer(inputs=y, num_filters=num_filters, activation=None)
            if stack>0 and res_block == 0:
                x=resnet_layer(inputs=x, num_filters=num_filters,kernel_size=1
                               ,strides=strides,activation=None,
                               batch_normalization=False)
            
            x=keras.layers.add([x,y])
            x=Activation('relu')(x)
        num_filters*=2
    x=AveragePooling2D(pool_size=8)(x)
    y=Flatten()(x)
    outputs=Dense(num_classes,
                  activation='softmax',
                  kernel_initializer='he_normal')(y)
    model=Model(inputs=inputs, outputs=outputs)
    return model

In [15]:
model=resnet_v1(input_shape=input_shape,depth=depth)
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(learning_rate=lr_schedule(0)),
              metrics=['accuracy'])
model.summary()
print(model_type)
save_dir=os.path.join(os.getcwd(),'saved_models')
model_name='cifar10_%s_model.{epoch:03d}.keras'% model_type
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
filepath=os.path.join(save_dir,model_name)

checkpoint = ModelCheckpoint(filepath=filepath,
                             monitor='val_acc',
                             verbose=1,
                             save_best_only=True)
lr_scheduler=LearningRateScheduler(lr_schedule)

lr_reducer=ReduceLROnPlateau(factor=np.sqrt(0.1),
                               cooldown=0,
                               patience=5,
                               min_lr=0.5e-6)

callbacks=[checkpoint, lr_scheduler, lr_reducer]



Learning rate:  0.001


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_2       │ (None, 32, 32, 3) │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_58 (Conv2D)  │ (None, 32, 32,    │        448 │ input_layer_2[0]… │
│                     │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 32, 32,    │         64 │ conv2d_58[0][0]   │
│ (BatchNormalizatio… │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_56       │ (None, 32, 32,    │          0 │ batch_normalizat… │
│ (Activation)        │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_59 (Conv2D)  │ (None, 32, 32,    │      2,320 │ activation_56[0]… │
│                     │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 32, 32,    │         64 │ conv2d_59[0][0]   │
│ (BatchNormalizatio… │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_57       │ (None, 32, 32,    │          0 │ batch_normalizat… │
│ (Activation)        │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_60 (Conv2D)  │ (None, 32, 32,    │      2,320 │ activation_57[0]… │
│                     │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 32, 32,    │         64 │ conv2d_60[0][0]   │
│ (BatchNormalizatio… │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_27 (Add)        │ (None, 32, 32,    │          0 │ activation_56[0]… │
│                     │ 16)               │            │ batch_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_58       │ (None, 32, 32,    │          0 │ add_27[0][0]      │
│ (Activation)        │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_61 (Conv2D)  │ (None, 32, 32,    │      2,320 │ activation_58[0]… │
│                     │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 32, 32,    │         64 │ conv2d_61[0][0]   │
│ (BatchNormalizatio… │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_59       │ (None, 32, 32,    │          0 │ batch_normalizat… │
│ (Activation)        │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_62 (Conv2D)  │ (None, 32, 32,    │      2,320 │ activation_59[0]… │
│                     │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 32, 32,    │         64 │ conv2d_62[0][0]   │
│ (BatchNormalizatio… │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_28 (Add)        │ (None, 32, 32,    │          0 │ activation_58[0]

 Total params: 861,770 (3.29 MB)

 Trainable params: 857,706 (3.27 MB)

 Non-trainable params: 4,064 (15.88 KB)

ResNet56v2


In [16]:
if not data_augmentation:
    print("Not using data augmentation.")
    model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(x_test, y_test),
              shuffle=True,
              callbacks=callbacks)
else:
    print("Using real-time data augmentation.")
    datagen=ImageDataGenerator(
        featurewise_center=False,
        samplewise_center=False,
        featurewise_std_normalization=False,
        samplewise_std_normalization=False,
        zca_whitening=False,
        zca_epsilon=1e-06,
        rotation_range=0,
        zoom_range=0.1,
        channel_shift_range=0.1,
        fill_mode='nearest',
        cval=0.,
        width_shift_range=0.1,
        height_shift_range=0.1,
        horizontal_flip=True,
        )
    datagen.fit(x_train)

    model.fit(datagen.flow(x_train, y_train, batch_size=batch_size),
              validation_data=(x_test,y_test),
              epochs=epochs,
              verbose=1,
              steps_per_epoch=x_train.shape[0],
              callbacks=callbacks)

# Evaluate the model
score = model.evaluate(x_test, y_test, verbose=1)

print('Test loss:', score[0])   
print('Test accuracy:', score[1])

Using real-time data augmentation.


c:\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Learning rate:  0.001
Epoch 1/20
  593/50000 ━━━━━━━━━━━━━━━━━━━━ 4:49:08 351ms/step - accuracy: 0.2768 - loss: 2.7987

KeyboardInterrupt: 